In [1]:
import os
import numpy as np
import tensorflow as tf

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from sklearn.model_selection import train_test_split

C:\Users\loicl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\loicl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\loicl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\loicl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
train_data_dir = r'..\..\spectro_fonctionnels\Train'
test_data_dir = r'..\..\spectro_fonctionnels\Test'
img_width, img_height = 224, 224

In [3]:
#os.chdir('..\..\test2')
os.getcwd()

'C:\\Users\\loicl\\Desktop\\cours_EFREI_L3\\semestre2\\summerCamp\\projet\\models'

In [4]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32 
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle = True,
        class_mode='binary')

test_generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

Found 5965 images belonging to 2 classes.
Found 1490 images belonging to 2 classes.


In [5]:
# On définit l'architecture du modèle :

model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# On compile le modeles :

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# On affiche le model :

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
activation_1 (Activation)    (None, 109, 109, 32)      0         
__________________________________________

In [7]:
# On définit les parametres pour l'entrainement :
epochs = 10
train_samples = 5965
test_samples = 1490

In [8]:
historique = model.fit_generator(
        train_generator,
        steps_per_epoch=train_samples // batch_size,
        epochs=epochs,
        validation_data = test_generator,
        validation_steps=test_samples// batch_size)

Epoch 1/10
186/186 [==============================] - 942s 5s/step - loss: 0.5779 - acc: 0.7617 - val_loss: 0.3907 - val_acc: 0.8166
Epoch 2/10
186/186 [==============================] - 455s 2s/step - loss: 0.3515 - acc: 0.8552 - val_loss: 0.3923 - val_acc: 0.8023
Epoch 3/10
186/186 [==============================] - 395s 2s/step - loss: 0.2986 - acc: 0.8839 - val_loss: 0.4000 - val_acc: 0.7833
Epoch 4/10
186/186 [==============================] - 326s 2s/step - loss: 0.2578 - acc: 0.9033 - val_loss: 0.2595 - val_acc: 0.9090
Epoch 5/10
186/186 [==============================] - 412s 2s/step - loss: 0.2354 - acc: 0.9093 - val_loss: 0.2364 - val_acc: 0.8974
Epoch 6/10
186/186 [==============================] - 323s 2s/step - loss: 0.2155 - acc: 0.9189 - val_loss: 0.1984 - val_acc: 0.9321
Epoch 7/10
186/186 [==============================] - 309s 2s/step - loss: 0.2070 - acc: 0.9273 - val_loss: 0.3196 - val_acc: 0.9103
Epoch 8/10
186/186 [==============================] - 315s 2s/step - 

In [9]:
model.save(r'save/premier_model_potable')